En este código se importa la base de datos "Datos_Climatologicos_Unificados.xlsx", que contiene el valor real de precipitación para cada estación desde 01-ene-1980 hasta 09-nov-2024. También se importa la base de datos "PronósticosWRF.xlsx", que contiene el pronóstico del modelo WRF para las diferentes estaciones en diferentes días. Finalmente se importa "Percentiles.xlsx", que contiene la media y los percentiles 1/3 y 2/3 para el conjunto de precipitaciones de la Climatología (1991-2020) en cada día del año en las diferentes estaciones meteorológicas.

Para cada estación y día en la base de datos "PronósticosWRF.xlsx", se va a buscar el valor de precipitación real correspondiente en "Datos_Climatologicos_Unificados.xlsx", y también se busca la media y percentiles en la base de datos "Percentiles.xlsx", con lo que se puede definir el valor para las variables objetivo:

- Anomalía de precipitación del día siguiente con respecto al promedio de la Climatología
- Clasificación binaria de la precipitación del día siguiente como =0 ó >0
- Clasificación de la precipitación del día siguiente en cotas percentiles

El valor de estas variables se inserta como nuevas columnas en el dataframe "PronósticosWRF.xlsx", el cual se exporta al final con el nombre "DataWRF.xlsx"

# IMPORTACIÓN DE DATOS Y LIBRERÍAS

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# Cargar datos desde Excel

climatologia_df = pd.read_excel('Percentiles.xlsx')  # Archivo con climatología (media y percentiles para cada estación en cada día del año)
Datos_Reales = pd.read_excel('Datos_Climatologicos_Unificados.xlsx') # Datos de precipitación reales
PronosticosWRF = pd.read_excel('PronósticosWRF.xlsx') # Contiene las tres variables objetivo predichas por WRF

# DEFINICIÓN DE FUNCIONES

In [ ]:
# Función para calcular anomalía de precipitación

def anom_prec(prec, media):
    anom = prec - media
    suma = anom + media
    if suma < 0:
        anom = -media
    return anom

In [ ]:
# Función de clasificación binaria de la lluvia como =0 ó >0

def Lluvia_binaria(prec):
    if prec == 0:
        return 0
    else:
        return 1

In [ ]:
# Función para clasificar por percentiles

def clasificar_percentiles(prec, p1_3, p2_3):
    if p1_3 == 0 and p2_3 == 0 and prec == 0:
        return 0
    else:
        if prec <= p1_3:
            return -1
        elif prec >= p2_3:
            return 1
        else:
            return 0

# PROCESAMIENTO DE DATOS

In [ ]:
DataWRF = PronosticosWRF.copy()

# Inicializar las nuevas columnas
DataWRF['percentil_Real'] = None
DataWRF['Lluvia_Binaria_Real'] = None
DataWRF['Anom_prec_mañana_Real'] = None

# Iterar sobre las filas de los pronósticos
for i, row in DataWRF.iterrows():
    
    # Obtener la estación y la fecha de la fila actual
    estacion = row['Estacion']
    fecha = row['Fecha']
    
    # Buscar el valor real de precipitación en Datos_Reales para esa estación y fecha
    precip_real = Datos_Reales[(Datos_Reales['FECHA'] == fecha) & (Datos_Reales['ESTACIÓN'] == estacion)]['PRECIPITACIÓN'].values
    
    precip_real = precip_real[0]
    
    # Buscar la media y los percentiles en climatologia_df para esa estación y fecha
    climatologia = climatologia_df[(climatologia_df['ESTACIÓN'] == estacion) & (climatologia_df['DiaDelAno'] == fecha.strftime('%m-%d'))]
    
    # Obtener la media y percentiles del dataframe climatologia_df
    media = climatologia['Media'].values[0]
    p1_3 = climatologia['Percentil_1_/3'].values[0]
    p2_3 = climatologia['Percentil_2/3'].values[0]
    
    # Ejecutar las tres funciones para clasificar y calcular anomalías
    anomalía = anom_prec(precip_real, media)
    clasificacion_percentiles = clasificar_percentiles(precip_real, p1_3, p2_3)
    Prec_binaria = Lluvia_binaria(precip_real)
    
    # Almacenar los resultados en las nuevas columnas de DataWRF
    DataWRF.at[i, 'percentil_Real'] = clasificacion_percentiles
    DataWRF.at[i, 'Lluvia_Binaria_Real'] = Prec_binaria
    DataWRF.at[i, 'Anom_prec_mañana_Real'] = anomalía

In [ ]:
# Exportar a Excel
DataWRF.to_excel('DataWRF.xlsx', index=False)